# Ergonomic pose fine-tuning: 

## fine tuning yolo for our classification task

- option 1: freeze the complete model, use it as a feature extractor and add a classification head to be trained on our dataset
- option 2: cut the keypoint output layer, freeze the rest of the model and add a classification head to be trained on our dataset

### 1) YOLOv8pose model as feature extractor

imports:

In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image
#%pip install -U ultralytics

### load the model

In [2]:
model = YOLO('yolov8n-pose.pt')

In [51]:
# todo: outcomment this to try if it works + model.eval() later
for param in model.parameters():
    param.requires_grad = False # freeze the YOLO model

#for name, param in model.named_parameters():
    #print(f"{name} is {'not ' if param.requires_grad else ''}frozen"

defining a classification layer:

In [3]:
import torch
import torch.nn as nn

# `feature_extractor` is the part of the model suitable for feature extraction

class YOLO_kp_Classifier(nn.Module):
    def __init__(self, num_keypoints, num_classes=3):
        super(YOLO_kp_Classifier, self).__init__()
        # to flatten the output 
        #self.flatten = nn.Flatten()
        # add new classification layer(s) to the model
        self.classifier = nn.Sequential(
            nn.Linear(num_keypoints*2, 512),  # 17 keypoints * 2 (x, y coordinates for each keypoint)
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )
    
    def forward(self, keypoints_2d):
        #keypoints_2d = self.flatten(keypoints_2d)
        output = self.classifier(keypoints_2d)
        return output


the keypoints are calculated in 2D and 3D, which we will leverage for our predictions

initializing the model:

In [4]:
# initialize the classifier model
num_classes = 3  #  3-class classification problem
num_keypoints = 17  # 17 keypoints in the model

kp_classifier_model = YOLO_kp_Classifier(num_keypoints=num_keypoints, num_classes=num_classes)

load the own dataset:

In [5]:
from torch.utils.data import random_split, DataLoader, Subset
from torchvision import datasets, transforms

In [6]:
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)), # Resize images for the model?
    transforms.ToTensor()
    #transforms.Normalize(), # todo: add a normalization step!
])

In [7]:
data_path = 'example_images'
own_dataset = datasets.ImageFolder(root=data_path, transform=data_transforms)
#type(own_dataset)

splitting the data into train, validation and test set:

In [8]:
generator1 = torch.Generator().manual_seed(13)  # set seed for reproducibility of the split
train_and_val_dataset, test_dataset =random_split(own_dataset, [0.8, 0.2], generator=generator1)  # 80% training and evaluation, 20% testing

#type(train_and_val_dataset)

feature extraction with yolo:

In [9]:
def get_keypoints_from_yolo(model, inputs):
    #model.eval()  # YOLO model set to evaluation mode because we are not training it
    with torch.no_grad(): # gradients are not computed for the frozen model
        results = model(inputs)
        keypoints = results[0].keypoints.data  # extract the keypoints from the results
    return keypoints

processing keypoints:

In [10]:
# todo: check if this is sufficient, otherwise try z-score normalization or other methods

# todo: !!!!!!!!! check correct position to use it / get min max earlier !!!!!!!!! needs to be the same normalization for all the data

def process_keypoints_for_classifier(keypoints):
    # processing the keypoints in a suitable format for the classifier
    # keypoints is of shape [batch_size, 17, 3]
    # normalizing keypoints across the keypoints dimension
    # with min-max normalization i
    kp_tensor_norm = (keypoints - keypoints.min(dim=1, keepdim=True)[0]) / (keypoints.max(dim=1, keepdim=True)[0] - keypoints.min(dim=1, keepdim=True)[0])
    
    # Flatten the last two dimensions while keeping the batch dimension
    batch_flattened = kp_tensor_norm.view(keypoints.size(0), -1)  # Reshapes to [batch_size, 17*3]
    return batch_flattened

#### training the model:

- with cross-entropy loss function (fits our classification task)
- Adam optimizer

HPO:

In [11]:
#!pip install --upgrade pip
#%pip install scikit-learn
#%pip install optuna
import optuna
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np

/Users/ari/Documents/Data_Science/3_semester/learning_from_images/egronomic_pose_project/ergonomic_pose_detect/ergonomic_pose.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# initialize the classifier model
num_classes = 3  #  3-class classification problem
num_keypoints = 17  # 17 keypoints in the model
# loss function and optimizer
criterion = nn.CrossEntropyLoss()

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # else ("mps" if torch.backends.mps.is_available())
print(f"Using device: {device}")

Using device: cpu


In [18]:
def train_and_eval_model(kp_classifier_model, model, optimizer, num_epochs, train_loader, val_loader, device):
    
    kp_classifier_model = kp_classifier_model.to(device)  # move model to device
    kp_classifier_model.train()  # set model to training mode
    
    print("type of train_loader", type(train_loader))

    for epoch in range(num_epochs):
        for batch in train_loader:
            # access images and labels
            inputs = batch[0].to(device)
            labels = batch[1].to(device)
            #print("inputs", type(inputs), len(inputs), inputs)
            #inputs, labels = inputs.to(device), labels.to(device)  # move data to device
            model = model.to(device) # YOLO move model to device
            optimizer.zero_grad()  # Zero the parameter gradients
            keypoints = get_keypoints_from_yolo(model, inputs) # get keypoints from the YOLO model
            print("keypoints", type(keypoints), len(keypoints[0]), keypoints)
            processed_kps = process_keypoints_for_classifier(keypoints) # prepare the keypoints for the classifier
            print("processed_kps", type(processed_kps), len(processed_kps[0]), processed_kps)
            classification_output = kp_classifier_model(processed_kps) # get results for the classification 
            loss = criterion(classification_output , labels)  # Compute loss
            loss.backward()  # Backpropagate the loss
            optimizer.step()  # Update weights

            # todo: further processing, such as calculating accuracy or loss, goes here

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
            
        
        kp_classifier_model.eval()  # model to evaluation mode

        total_loss = 0
        all_predictions = []
        all_labels = []

        with torch.no_grad():  # no need to compute gradients, because we are in evaluation mode
            for inputs, labels in val_loader:  # iterate over validation dataset
                inputs, labels = inputs.to(device), labels.to(device)  # move data to device
                keypoints = get_keypoints_from_yolo(model, inputs) # get keypoints from the YOLO model
                processed_kps = process_keypoints_for_classifier(keypoints) # prepare the keypoints for the classifier
                classification_output = kp_classifier_model(processed_kps) # get results for the classification 
                loss = criterion(classification_output , labels)  # compute loss
                total_loss += loss.item()  # accumulate the loss
                # get predictions for output
                _, predicted = torch.max(classification_output.data, 1)
                # collect the predictions and labels
                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
            # calculate the validation metrics:
            avg_loss = total_loss / len(val_loader)  # get the average loss
            accuracy = (np.array(all_predictions) == np.array(all_labels)).mean()
            f1 = f1_score(all_labels, all_predictions, average='weighted')  
            conf_matrix = confusion_matrix(all_labels, all_predictions)
            
            print(f"Validation Loss: {avg_loss:.4f}")
            print(f"Validation Accuracy: {accuracy:.4f}")
            print(f"Validation F1 Score: {f1:.4f}")
            print("Confusion Matrix:")
            print(conf_matrix)
            
            # here: F1 score chosen as the metric to optimize
            # other options: - combining metrics like accuracy and F1 score to maximize on both
            #                        - or multi-objective optimization on F1 score and accuracy
    return f1 

In [15]:
# this function was generated by gpt-4

def get_k_fold_indices(n, k=5, random_seed=None):
    """
    Generate indices for k-fold cross-validation.

    Parameters:
    - n: Total number of samples in the dataset.
    - k: Number of folds.
    - random_seed: Optional seed for reproducibility.

    Returns:
    - A list of tuples, each containing (train_indices, val_indices) for a fold.
    """
    # Initialize the random generator
    g = torch.Generator()
    if random_seed is not None:
        g.manual_seed(random_seed)
    
    # Generate a random permutation of indices
    indices = torch.randperm(n, generator=g).tolist()
    
    # Calculate fold sizes
    fold_sizes = [n // k for _ in range(k)]
    for i in range(n % k):
        fold_sizes[i] += 1
    
    # Generate train and validation indices for each fold
    current = 0
    folds_indices = []
    for fold_size in fold_sizes:
        start, end = current, current + fold_size
        val_indices = indices[start:end]
        train_indices = indices[:start] + indices[end:]
        folds_indices.append((train_indices, val_indices))
        current = end
    
    return folds_indices


In [16]:
def objective(trial):
    # Define hyperparameters to optimize
    lr = trial.suggest_categorical("lr", [1e-4, 5e-4, 1e-3])
    #momentum = trial.suggest_categorical("momentum", [0.9, 0.95])
    batch_size = trial.suggest_categorical("batch_size", [4, 8, 16])
    num_epochs = trial.suggest_categorical("num_epochs", [100, 200, 300])

    # convert dataset into a list for index-based access
    dataset = train_and_val_dataset

    validation_scores = []

    n = len(dataset)
    k = 5
    folds_indices = get_k_fold_indices(n, k, random_seed=13)

    for fold, (train_idx, val_idx) in enumerate(folds_indices, start=1):
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)
        print("type of train subset: " , type(train_subset))
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        print("type of train loader: " , type(train_loader))
        # initializing the model and optimizer with the chosen hyperparameters
        kp_classifier_model = YOLO_kp_Classifier(num_keypoints=num_keypoints, num_classes=num_classes)
        optimizer = torch.optim.Adam(kp_classifier_model.classifier.parameters(), lr=lr, weight_decay=1e-5)
    
        # training and evaluating the model
        validation_score = train_and_eval_model(kp_classifier_model, model, optimizer, num_epochs, train_loader, val_loader, device)
        validation_scores.append(validation_score)

    return np.mean(validation_scores)


In [52]:
#-----------------------------------------------------------------------------------------------------------
        #  todo!! there is a bug --> YOLO downloads its own data to use...etc. #
#-----------------------------------------------------------------------------------------------------------

In [19]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # run 20 trials

# Best hyperparameters
print("Best hyperparameters:", study.best_params)

[I 2024-03-16 14:39:37,596] A new study created in memory with name: no-name-098501ca-1105-49d3-a752-e7970a034401


type of train subset:  <class 'torch.utils.data.dataset.Subset'>
type of train loader:  <class 'torch.utils.data.dataloader.DataLoader'>
type of train_loader <class 'torch.utils.data.dataloader.DataLoader'>

0: 224x224 1 person, 1: 224x224 1 person, 2: 224x224 1 person, 3: 224x224 1 person, 4: 224x224 1 person, 60.7ms
Speed: 0.0ms preprocess, 12.1ms inference, 0.5ms postprocess per image at shape (1, 3, 224, 224)


[W 2024-03-16 14:39:38,794] Trial 0 failed with parameters: {'lr': 0.0001, 'batch_size': 8, 'num_epochs': 200} because of the following error: RuntimeError('mat1 and mat2 shapes cannot be multiplied (1x51 and 34x512)').
Traceback (most recent call last):
  File "/Users/ari/Documents/Data_Science/3_semester/learning_from_images/egronomic_pose_project/ergonomic_pose_detect/ergonomic_pose.venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/9c/8w6dd7612jv4k22v8dfgbq040000gn/T/ipykernel_14326/3745596659.py", line 29, in objective
    validation_score = train_and_eval_model(kp_classifier_model, model, optimizer, num_epochs, train_loader, val_loader, device)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/9c/8w6dd7612jv4k22v8dfgbq040000gn/T/ipykernel_14326/274171.py", lin

keypoints <class 'torch.Tensor'> 17 tensor([[[1.2443e+02, 5.7125e+01, 9.7429e-01],
         [1.3540e+02, 5.5069e+01, 9.5667e-01],
         [1.2256e+02, 5.0324e+01, 9.2401e-01],
         [1.5067e+02, 6.6387e+01, 8.1318e-01],
         [1.1728e+02, 5.4056e+01, 6.0656e-01],
         [1.5255e+02, 9.7447e+01, 9.7892e-01],
         [9.1749e+01, 8.5165e+01, 9.6479e-01],
         [1.5547e+02, 1.3127e+02, 9.1782e-01],
         [5.5499e+01, 1.2696e+02, 8.2078e-01],
         [1.3548e+02, 9.2259e+01, 8.8759e-01],
         [8.8665e+01, 1.3739e+02, 7.8881e-01],
         [1.3879e+02, 1.7092e+02, 7.1291e-01],
         [9.7614e+01, 1.6867e+02, 6.3443e-01],
         [0.0000e+00, 0.0000e+00, 1.3601e-01],
         [0.0000e+00, 0.0000e+00, 9.7912e-02],
         [0.0000e+00, 0.0000e+00, 2.4750e-02],
         [0.0000e+00, 0.0000e+00, 1.8985e-02]]])
processed_kps <class 'torch.Tensor'> 51 tensor([[0.8004, 0.3342, 0.9952, 0.8709, 0.3222, 0.9768, 0.7883, 0.2944, 0.9428, 0.9692, 0.3884, 0.8273, 0.7544, 0.3163, 0.

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x51 and 34x512)

In [ ]:
best_params = study.best_params
# extract best_parameters
batch_size = best_params['batch_size']
lr = best_params['lr']
momentum = best_params['momentum']
num_epochs = best_params['num_epochs']

#### train and evaluate final model on test set:

Unfortunately Optuna cannot output the best model, so we train again on the combined train and validation set with the best parameters found.

dataloader for final evaluation:

In [ ]:
train_and_eval_loader = DataLoader(train_and_val_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

final evaluation on test set:

In [ ]:
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(kp_classifier_model.classifier.parameters(), lr=lr, momentum=momentum, weight_decay=1e-5)

# new instance of the model:
kp_classifier_model = YOLO_kp_Classifier(num_keypoints=num_keypoints, num_classes=num_classes)
kp_classifier_model = kp_classifier_model.to(device)  # move model to device
kp_classifier_model.train()  # set model to training mode

for epoch in range(num_epochs):
    for inputs, labels in train_and_eval_loader:  # Assuming data_loader is your DataLoader instance for the dataset
        inputs, labels = inputs.to(device), labels.to(device)  # move data to device
        optimizer.zero_grad()  # Zero the parameter gradients
        keypoints = get_keypoints_from_yolo(model, inputs) # get keypoints from the YOLO model
        processed_kps = process_keypoints_for_classifier(keypoints) # prepare the keypoints for the classifier
        classification_output = kp_classifier_model(processed_kps) # get results for the classification 
        loss = criterion(classification_output , labels)  # Compute loss
        loss.backward()  # Backpropagate the loss
        optimizer.step()  # Update weights

        # todo: further processing, such as calculating accuracy or loss, goes here

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        
    
    kp_classifier_model.eval()  # model to evaluation mode

    total_loss = 0
    all_predictions = []
    all_labels = []
    with torch.no_grad():  # no need to compute gradients, because we are in evaluation mode
        for inputs, labels in test_loader:  # iterate over validation dataset
            inputs, labels = inputs.to(device), labels.to(device)  # move data to device
            keypoints = get_keypoints_from_yolo(model, inputs) # get keypoints from the YOLO model
            processed_kps = process_keypoints_for_classifier(keypoints) # prepare the keypoints for the classifier
            classification_output = kp_classifier_model(processed_kps) # get results for the classification 
            loss = criterion(classification_output , labels)  # compute loss
            total_loss += loss.item()  # accumulate the loss
            _, predicted = torch.max(classification_output.data, 1)
            # collect the predictions and labels
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
        # calculate the validation metrics:
        avg_loss = total_loss / len(test_loader)  # get the average loss
        accuracy = (np.array(all_predictions) == np.array(all_labels)).mean()
        f1 = f1_score(all_labels, all_predictions, average='weighted')  
        conf_matrix = confusion_matrix(all_labels, all_predictions)
        
        print(f"Test Loss: {avg_loss:.4f}")
        print(f"Test Accuracy: {accuracy:.4f}")
        print(f"Test F1 Score: {f1:.4f}")
        print("Confusion Matrix:")
        print(conf_matrix)




In [20]:
import torch

# Define your original tensor
tensor = torch.tensor([[[1.2443e+02, 5.7125e+01, 9.7429e-01],
                        [1.3540e+02, 5.5069e+01, 9.5667e-01],
                        [1.2256e+02, 5.0324e+01, 9.2401e-01],
                        [1.5067e+02, 6.6387e+01, 8.1318e-01],
                        [1.1728e+02, 5.4056e+01, 6.0656e-01],
                        [1.5255e+02, 9.7447e+01, 9.7892e-01],
                        [9.1749e+01, 8.5165e+01, 9.6479e-01],
                        [1.5547e+02, 1.3127e+02, 9.1782e-01],
                        [5.5499e+01, 1.2696e+02, 8.2078e-01],
                        [1.3548e+02, 9.2259e+01, 8.8759e-01],
                        [8.8665e+01, 1.3739e+02, 7.8881e-01],
                        [1.3879e+02, 1.7092e+02, 7.1291e-01],
                        [9.7614e+01, 1.6867e+02, 6.3443e-01],
                        [0.0000e+00, 0.0000e+00, 1.3601e-01],
                        [0.0000e+00, 0.0000e+00, 9.7912e-02],
                        [0.0000e+00, 0.0000e+00, 2.4750e-02],
                        [0.0000e+00, 0.0000e+00, 1.8985e-02]]])

# Remove the third column
modified_tensor = tensor[:, :, :2]

# Print or use the modified tensor
print(modified_tensor)


tensor([[[124.4300,  57.1250],
         [135.4000,  55.0690],
         [122.5600,  50.3240],
         [150.6700,  66.3870],
         [117.2800,  54.0560],
         [152.5500,  97.4470],
         [ 91.7490,  85.1650],
         [155.4700, 131.2700],
         [ 55.4990, 126.9600],
         [135.4800,  92.2590],
         [ 88.6650, 137.3900],
         [138.7900, 170.9200],
         [ 97.6140, 168.6700],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000],
         [  0.0000,   0.0000]]])
